# 【手撕LLM-Mixtral】

![mistral](./mistral.png)

In [3]:
!export HF_ENDPOINT=https://hf-mirror.com

## 1. 创建Mixtral 模型

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import torch.nn.functional as F
seed_value = 42
torch.manual_seed(seed_value)

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

# 创建小模型用于调试
config = AutoConfig.from_pretrained(model_id)
config.num_hidden_layers = 2
config.num_attention_heads = 8
config.hidden_size = 128
config.intermediate_size = 256
config.intermediate_size = config.hidden_size*2
config.num_experts_per_tok = 2 # Top-2 专家数量
config.num_local_experts = 8   # 专家总数量
# print(config)

model =  AutoModelForCausalLM.from_config(config)
print(model)

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 128)
    (layers): ModuleList(
      (0-1): 2 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=128, out_features=128, bias=False)
          (k_proj): Linear(in_features=128, out_features=128, bias=False)
          (v_proj): Linear(in_features=128, out_features=128, bias=False)
          (o_proj): Linear(in_features=128, out_features=128, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=128, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBLockSparseTop2MLP(
              (w1): Linear(in_features=128, out_features=256, bias=False)
              (w2): Linear(in_features=256, out_features=128, bias=False)
              (w3): Linear(in_features=128, out_features=256, bias=False)
              (act_fn): SiLU()
            )
          )
        )
        (input_layernorm): MixtralRMSNorm()
        (post_attention_layernorm): MixtralRMSNorm()
      )
    )
    (norm): MixtralRMSNorm()
  )
  (lm_head): Linear(in_features=128, out_features=32000, bias=False)
)

## 创建MoE层

In [17]:
import torch
from torch import nn
from transformers import MixtralConfig

class MixtralBLockSparseTop2MLP(nn.Module):
    def __init__(self, config: MixtralConfig):
        super().__init__()
        self.ffn_dim = config.intermediate_size
        self.hidden_dim = config.hidden_size

        self.w1 = nn.Linear(self.hidden_dim, self.ffn_dim, bias=False)
        self.w2 = nn.Linear(self.ffn_dim, self.hidden_dim, bias=False)
        self.w3 = nn.Linear(self.hidden_dim, self.ffn_dim, bias=False)

        self.act_fn = nn.SiLU()

    def forward(self, hidden_states):
        y = self.act_fn(self.w1(hidden_states)) * self.w3(hidden_states)
        y = self.w2(y)
        return y

x = torch.randn(1, 64, 128)
expert = MixtralBLockSparseTop2MLP(config)
print('单个专家为原LLaMA的MLP层')
print(expert)
g = expert(x)
print('单个专家输入:', x.shape)
print('单个专家输出结果：', g.shape)


单个专家为原LLaMA的MLP层

MixtralBLockSparseTop2MLP(
  (w1): Linear(in_features=128, out_features=256, bias=False)
  (w2): Linear(in_features=256, out_features=128, bias=False)
  (w3): Linear(in_features=128, out_features=256, bias=False)
  (act_fn): SiLU()
)

单个专家输入:
torch.Size([1, 64, 128])

单个专家输出结果：
torch.Size([1, 64, 128])

## 创建混合专家

![mixtral-mode](./moe.png)

In [18]:
class MixtralSparseMoeBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.hidden_dim = config.hidden_size
        self.ffn_dim = config.intermediate_size
        self.num_experts = config.num_local_experts
        self.top_k = config.num_experts_per_tok

        # gating
        self.gate = nn.Linear(self.hidden_dim, self.num_experts, bias=False)

        self.experts = nn.ModuleList([MixtralBLockSparseTop2MLP(config) \
                                      for _ in range(self.num_experts)])

x = torch.randn(1, 64, 128)
experts = MixtralSparseMoeBlock(config)
print('多个专家混合专家')
print(experts)
# print
# gs = experts(x)
# print('多个专家输入:', x.shape)
# print('多个专家混合输出结果：', gs.shape)

多个专家混合专家

MixtralSparseMoeBlock(
  (gate): Linear(in_features=128, out_features=8, bias=False)
  (experts): ModuleList(
    (0-7): 8 x MixtralBLockSparseTop2MLP(
      (w1): Linear(in_features=128, out_features=256, bias=False)
      (w2): Linear(in_features=256, out_features=128, bias=False)
      (w3): Linear(in_features=128, out_features=256, bias=False)
      (act_fn): SiLU()
    )
  )
)

## 混合专家 Forward

In [55]:
# 阶段一
# 计算稀疏 gating 值
tokens = 6
x = torch.randn(1, tokens, 128) # 6个token
hidden_states = x
batch_size, sequence_length, hidden_dim = hidden_states.shape
hidden_states = hidden_states.view(-1, hidden_dim)

 # 每层都会产生router_logits, 将用于最后作 load balance loss
router_logits = experts.gate(hidden_states)
print(f'experts.gate output router logits : \n {router_logits}')

# 计算 TopK 的 专家 logits 和 Top2 专家的位置
routing_weights = F.softmax(router_logits, dim=1, dtype=torch.float)
print(f'softmax weight  : \n {routing_weights}')

routing_weights, selected_experts = torch.topk(routing_weights, \
                                               experts.top_k, dim=-1)
print(f'expert select : \n {selected_experts}')
print(f'topk : \n {routing_weights}')

routing_weights /= routing_weights.sum(dim=-1, keepdim=True)
print(f'topk归一化 : \n {routing_weights}')

routing_weights = routing_weights.to(hidden_states.dtype)

## One Hot 编码
expert_mask = torch.nn.functional.one_hot(selected_experts, \
                                          num_classes=experts.num_experts).permute(2, 1, 0)
for i in range(tokens):
    print(f'【x_{i}】\n', expert_mask[:,:,i]) 

experts.gate output router logits : 
 tensor([[-0.2740, -1.0367,  1.0243, -0.6972, -0.3429,  0.2973,  0.5766, -1.0476],
        [ 0.4899, -0.0653,  0.1691, -0.0317,  0.4138, -0.5443, -0.2966,  0.2359],
        [-1.2784,  0.9846, -0.7407,  0.2706, -0.4702,  0.3019, -1.1157, -0.7304],
        [-0.0710, -0.0827,  0.5985,  1.3902, -0.2018,  0.4387, -0.7845,  0.4170],
        [ 0.8758,  0.0289,  0.3279, -0.6008,  0.7846,  0.2894, -0.1772,  0.0935],
        [-0.0115, -0.0402,  1.1442,  0.6230,  0.2707, -0.6150,  0.2534,  0.1566]],
       grad_fn=<MmBackward0>)

softmax weight  : 
 tensor([[0.0886, 0.0413, 0.3244, 0.0580, 0.0827, 0.1568, 0.2073, 0.0409],
        [0.1850, 0.1062, 0.1342, 0.1098, 0.1714, 0.0658, 0.0842, 0.1435],
        [0.0370, 0.3555, 0.0633, 0.1741, 0.0830, 0.1796, 0.0435, 0.0640],
        [0.0774, 0.0765, 0.1512, 0.3338, 0.0679, 0.1289, 0.0379, 0.1262],
        [0.2217, 0.0950, 0.1282, 0.0506, 0.2024, 0.1233, 0.0774, 0.1014],
        [0.0878, 0.0853, 0.2788, 0.1655, 0.1164, 0.0480, 0.1144, 0.1038]],
       grad_fn=<SoftmaxBackward0>)

expert select : 
 tensor([[2, 6],
        [0, 4],
        [1, 5],
        [3, 2],
        [0, 4],
        [2, 3]])

topk : 
 tensor([[0.3244, 0.2073],
        [0.1850, 0.1714],
        [0.3555, 0.1796],
        [0.3338, 0.1512],
        [0.2217, 0.2024],
        [0.2788, 0.1655]], grad_fn=<TopkBackward0>)

topk归一化 : 
 tensor([[0.6101, 0.3899],
        [0.5190, 0.4810],
        [0.6643, 0.3357],
        [0.6882, 0.3118],
        [0.5228, 0.4772],
        [0.6274, 0.3726]], grad_fn=<DivBackward0>)

...

【x_1】
 tensor([[1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 1],
        [0, 0],
        [0, 0],
        [0, 0]])

【x_2】
 tensor([[0, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 1],
        [0, 0],
        [0, 0]])

【x_3】
 tensor([[0, 0],
        [0, 0],
        [0, 1],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]])

【x_4】
 tensor([[1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 1],
        [0, 0],
        [0, 0],
        [0, 0]])

【x_5】
 tensor([[0, 0],
        [0, 0],
        [1, 0],
        [0, 1],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]])

In [56]:
## 最终结果
final_hidden_states = torch.zeros(
    (batch_size * sequence_length, hidden_dim), \
        dtype=hidden_states.dtype, device=hidden_states.device
)
print(f'final moe result shape for each token: {final_hidden_states.shape}')

# 每个专家收集需要计算token
for expert_idx in range(experts.num_experts):

    print(f'----------------- expert {expert_idx} --------------------')

    expert_layer = experts.experts[expert_idx]
    print(expert_mask[expert_idx])
    idx, top_x = torch.where(expert_mask[expert_idx])
    print(f'专家 {expert_idx} 选到的样本编号:',top_x.tolist())  # select x_idx for expert top1
    print(f'专家 {expert_idx} top1:0, top2:1 ',idx.tolist())    # 0 is top1 ,1 is top2
    print(f'有 {len(top_x)} / {x.shape[1]} token 选到专家 {expert_idx}')
    
    top_x_list = top_x.tolist()
    idx_list = idx.tolist()

    current_state = hidden_states[None, top_x_list].reshape(-1, hidden_dim)

    # expert_0(x) * routing_weights
    current_hidden_states = expert_layer(current_state)  \
                            * routing_weights[top_x_list, idx_list, None]

    # 将计算的单个专家结果填入到结果表里
    final_hidden_states.index_add_(0, top_x, current_hidden_states.to(hidden_states.dtype))

    print(current_state.shape) 
    print(routing_weights[top_x_list, idx_list, None].shape)
    print(current_hidden_states.shape)
    print(final_hidden_states.shape)

    # if expert_idx == 1: break


final moe result shape for each token: torch.Size([6, 128])

----------------- expert 0 --------------------

tensor([[0, 1, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0]])

专家 0 选到的样本编号:
[1, 4]

专家 0 top1:0, top2:1 
[0, 0]

...

有 0 / 6 token 选到专家 7

torch.Size([0, 128])

torch.Size([0, 1])

torch.Size([0, 128])

torch.Size([6, 128])

## load balance loss

![moe-math](./moe_math.png)

In [70]:
import torch

num_experts = 8
batch = 10 
seq_length = 6
top_k = 2

print(f'sMoE num_experts:{num_experts} top_k:{top_k} batch:{batch} seq_length:{seq_length}')

router_logits_1 = torch.randn(batch, seq_length, num_experts).view(-1,num_experts) # layer 1
router_logits_2 = torch.randn(batch, seq_length, num_experts).view(-1,num_experts) # layer 2
router_logits = [router_logits_1, router_logits_2] 

concatenated_gate_logits = torch.cat(router_logits, dim = 0)
print('单层gating的路由logits:', router_logits_1.shape) 
print('两层gating的路由logits:', concatenated_gate_logits.shape)

print('根据logits top-k 计算热独编码')
routing_weights = torch.nn.functional.softmax(concatenated_gate_logits, dim=-1)
_, selected_experts = torch.topk(routing_weights, top_k, dim=-1)
expert_mask = torch.nn.functional.one_hot(selected_experts, num_experts)
print(expert_mask.shape)

tokens_sum_expert = torch.sum(expert_mask.float(), dim=0)
tokens_per_expert = torch.mean(expert_mask.float(), dim=0)
print(f'top1 每个专家平均处理的token   :', tokens_sum_expert[0])
print(f'top2 每个专家平均处理的token fi:', tokens_per_expert[1])
print(f'top1与top2水平合计', tokens_per_expert.sum(dim=1))

# Compute the average probability of routing to these experts
router_prob_per_expert = torch.mean(routing_weights, dim=0)
print('router_prob_per_expert Pi: ' , router_prob_per_expert)

print( '每个专家的负载：',  tokens_per_expert * router_prob_per_expert.unsqueeze(0))
overall_loss = torch.sum(tokens_per_expert * router_prob_per_expert.unsqueeze(0))
print('final loss:', overall_loss)


sMoE num_experts:8 top_k:2 batch:10 seq_length:6

单层gating的路由logits:
torch.Size([60, 8])

两层gating的路由logits:
torch.Size([120, 8])

根据logits top-k 计算热独编码

torch.Size([120, 2, 8])

...

top2 每个专家平均处理的token fi: tensor([0.1667, 0.1333, 0.1833, 0.0833, 0.1167, 0.1500, 0.0667, 0.1000])

top1与top2水平合计 tensor([1., 1.])

router_prob_per_expert Pi:  tensor([0.1236, 0.1184, 0.1351, 0.1168, 0.1311, 0.1147, 0.1156, 0.1447])

每个专家的负载： tensor([[0.0103, 0.0138, 0.0214, 0.0165, 0.0153, 0.0086, 0.0164, 0.0241],
        [0.0206, 0.0158, 0.0248, 0.0097, 0.0153, 0.0172, 0.0077, 0.0145]])

final loss: tensor(0.2520)